In [1]:
#import packages
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
main = pd.read_csv('D:/Downloads/Fifa Master data/player_id_table.csv')
positions = pd.read_csv("D:/Downloads/Fifa Master data/player_positions.csv")
nationality = pd.read_csv('D:/Downloads/Fifa Master data/national_team.csv')
scout_league = pd.read_csv('D:/Downloads/Fifa Master data/scout_league.csv')
player_stats = pd.read_csv('D:/Downloads/Fifa Master data/player_stats.csv')
player_position_stats = pd.read_csv('D:/Downloads/Fifa Master data/player_position_stats.csv')
club_info = pd.read_csv('D:/Downloads/Fifa Master data/club_info.csv')
main.head()

,player_id,short_name,long_name,position_id,club_team_id,nationality_id,nation_team_id
0,165153,K. Benzema,Karim Benzema,4,243,18,NaN
1,175943,D. Mertens,Dries Mertens,4,325,7,1325.0
2,216547,Rafa,Rafael Alexandre Fernandes Ferreira da Silva,4,234,38,NaN
3,210021,H. Mukhtar,Hany Mukhtar,4,114162,21,NaN
4,180826,Luiz Adriano,Luiz Adriano de Souza da Silva,4,741,54,NaN


In [3]:
filtered_df = main[main['nation_team_id'].isna()]
filtered_df.head()

,player_id,short_name,long_name,position_id,club_team_id,nationality_id,nation_team_id
0,165153,K. Benzema,Karim Benzema,4,243,18,NaN
2,216547,Rafa,Rafael Alexandre Fernandes Ferreira da Silva,4,234,38,NaN
3,210021,H. Mukhtar,Hany Mukhtar,4,114162,21,NaN
4,180826,Luiz Adriano,Luiz Adriano de Souza da Silva,4,741,54,NaN
5,213418,C. Akpom,Chuba Akpom,4,12,14,NaN


In [5]:
# list of all the nations that we can scout for
nationality.head()
nationality = nationality.sort_values(by = 'nationality_name')
country_list = nationality['nationality_name'].tolist()
print(country_list)

['Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Belarus', 'Belgium', 'Benin', 'Bermuda', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cameroon', 'Canada', 'Cape Verde Islands', 'Central African Republic', 'Chad', 'Chile', 'China PR', 'Colombia', 'Comoros', 'Congo', 'Congo DR', 'Costa Rica', 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', "Côte d'Ivoire", 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guinea Bissau', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'Iran', 'Iraq', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Kazakhstan', 'Kenya', 'Korea DPR', 'Korea Republic', 'Kosovo', 'Kuwait', 'Laos', 'Latvia', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxe

In [6]:
#scout country is the country we will scout for
scout_country = nationality[nationality['nationality_name']=='England']
scout_country.head()

,nationality_id,nationality_name
8,14,England


In [7]:
scout_candidates = filtered_df.merge(scout_country, on = ['nationality_id']) #filtering to selected country by inner join merge
scout_candidates.head()

,player_id,short_name,long_name,position_id,club_team_id,nationality_id,nation_team_id,nationality_name
0,213418,C. Akpom,Chuba Akpom,4,12,14,NaN,England
1,203502,B. Reid,Bobby De Cordova-Reid,4,1961,14,NaN,England
2,219216,J. Windass,Joshua Dean Windass,4,86,14,NaN,England
3,204289,S. Berahino,Saido Berahino,4,109,14,NaN,England
4,202052,B. Afobe,Benik Afobe,4,1943,14,NaN,England


In [8]:
scout_candidates = scout_candidates[scout_candidates['player_id']==213418]
scout_candidates.head()

,player_id,short_name,long_name,position_id,club_team_id,nationality_id,nation_team_id,nationality_name
0,213418,C. Akpom,Chuba Akpom,4,12,14,NaN,England


In [24]:
country = 'England'

In [26]:
scout_country = nationality[nationality['nationality_name']==country]
scout_candidates = filtered_df.merge(scout_country, on = ['nationality_id'])
#what position do they play?
scout_candidates = scout_candidates.merge(positions, on = ['position_id'])
#how good is our player overall?
player_stats = player_stats.drop_duplicates(subset = ['player_id'])
scout_candidates = scout_candidates.merge(player_stats, on = ['player_id'])
#add in how good is the player at that position
player_position_stats = player_position_stats.drop_duplicates(subset = ['player_id'])
melted_df = pd.melt(player_position_stats, id_vars = ['player_id'], var_name ='player_position',value_name='position_score')
melted_df['player_position'] = melted_df['player_position'].str.upper()
melted_df['position_score'] = melted_df['position_score'].str.extractall('(\d+)').astype(int).sum(level=0)
scout_candidates = scout_candidates.merge(melted_df, on = ['player_id','player_position'])
#where does this person play?
scout_candidates = scout_candidates.merge(club_info, on = ['club_team_id'])
scout_candidates = scout_candidates.merge(scout_league, on = ['league_id'])
scout_candidates.head(100)

,player_id,short_name,long_name,position_id,club_team_id,nationality_id,nation_team_id,nationality_name,player_position,preferred_foot,overall,work_rate,pace,shooting,passing,dribbling,defending,physic,position_score,club_name,league_id,league_name,scout_country
0,213418,C. Akpom,Chuba Akpom,4,12,14,NaN,England,CF,Right,72,Medium/Medium,83.0,71.0,53.0,71.0,29.0,74.0,70,Middlesbrough,14,Championship,England
1,204246,M. Bettinelli,Marcus Bettinelli,6,12,14,NaN,England,GK,Right,71,Medium/Medium,NaN,NaN,NaN,NaN,NaN,NaN,72,Middlesbrough,14,Championship,England
2,221982,P. Roberts,Patrick Roberts,13,12,14,NaN,England,RW,Left,71,Medium/Low,76.0,64.0,68.0,79.0,27.0,39.0,72,Middlesbrough,14,Championship,England
3,211504,L. Baker,Lewis Baker,1,12,14,NaN,England,CAM,Right,72,High/Medium,66.0,71.0,72.0,71.0,66.0,71.0,73,Middlesbrough,14,Championship,England
4,243608,R. Giles,Ryan Giles,7,12,14,NaN,England,LB,Left,71,High/Medium,89.0,56.0,67.0,69.0,58.0,64.0,69,Middlesbrough,14,Championship,England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,193884,S. Clucas,Sam Clucas,8,1960,14,NaN,England,LM,Left,71,Medium/Medium,66.0,70.0,71.0,69.0,66.0,72.0,71,Swansea City,14,Championship,England
96,186598,K. Naughton,Kyle Naughton,7,1960,14,NaN,England,LB,Right,71,Medium/Medium,68.0,52.0,64.0,67.0,70.0,68.0,71,Swansea City,14,Championship,England
97,212118,M. Grimes,Matt Grimes,5,1960,14,NaN,England,CM,Left,73,Medium/High,62.0,59.0,75.0,71.0,65.0,67.0,74,Swansea City,14,Championship,England
98,183774,R. Bennett,Ryan Bennett,2,1960,14,NaN,England,CB,Right,71,Low/Medium,36.0,25.0,46.0,36.0,72.0,71.0,71,Swansea City,14,Championship,England


In [29]:
def etl_pipeline_pt1(client_nation, position = ['position']):    
    # pull a country to scout
    scout_country = nationality[nationality['nationality_name']==client_nation]
    #filter down to players from that country
    scout_candidates = filtered_df.merge(scout_country, on = ['nationality_id'])
    #get player positions
    scout_candidates = scout_candidates.merge(positions, on = ['position_id'])
    if position is None:
        scout_candidates = scout_candidates
    else:
        scout_candidates = scout_candidates[scout_candidates['player_position'].isin(position)]
    #get player stats
    scout_candidates = scout_candidates.merge(player_stats, on = ['player_id'])
    melted_df = pd.melt(player_position_stats, id_vars = ['player_id'], var_name ='player_position',value_name='position_score')
    melted_df['player_position'] = melted_df['player_position'].str.upper()
    melted_df['position_score'] = melted_df['position_score'].str.extractall('(\d+)').astype(int).sum(level=0)
    #merge to main dataset
    scout_candidates = scout_candidates.merge(melted_df, on = ['player_id','player_position'])
    #get club info
    scout_candidates = scout_candidates.merge(club_info, on = ['club_team_id'])
    #get country of where to scout them
    scout_candidates = scout_candidates.merge(scout_league, on = ['league_id'])
    #group final report
    scout_report = scout_candidates.groupby(['scout_country','player_position']).agg(Average_Score=('position_score', 'mean'), Row_Count=('player_id', 'count')).reset_index()
    scout_report = scout_report.sort_values(by=['Row_Count', 'Average_Score'], ascending=[False, False])
    scout_report.head(100)
    return scout_candidates,scout_report

def etl_pipeline_pt2(df, scout_country=['country'], position = ['position']):
    df = df[df['scout_country'].isin(scout_country)]
    if position is None:
        df = df
    else:
        df = df[df['player_position'].isin(position)]
    df = df.sort_values(by=['position_score', 'club_name'], ascending=[False, True])
    df = df[['club_name','long_name','player_position','work_rate','pace','shooting','passing','dribbling','defending','physic','position_score','preferred_foot']]
    return df

In [28]:
candidates,report = etl_pipeline_pt1('England', position = None)
report.head(100)

,scout_country,player_position,Average_Score,Row_Count
8,England,LB,73.225806,31
2,England,CAM,74.500000,30
16,England,ST,74.925926,27
14,England,RW,73.541667,24
12,England,RB,73.619048,21
3,England,CB,73.000000,18
7,England,GK,74.117647,17
4,England,CDM,73.583333,12
9,England,LM,74.000000,11
6,England,CM,72.333333,6


In [30]:
candidates = etl_pipeline_pt2(candidates,scout_country=['England'], position = ['ST','RW','RB'])
candidates.head()

,club_name,long_name,player_position,work_rate,pace,shooting,passing,dribbling,defending,physic,position_score,preferred_foot
192,Chelsea,Reece James,RB,High/Medium,81.0,72.0,83.0,82.0,81.0,82.0,84,Right
158,Manchester United,Marcus Rashford,ST,Medium/Medium,88.0,82.0,76.0,83.0,42.0,73.0,84,Right
107,Leicester City,Jamie Vardy,ST,High/Medium,84.0,82.0,70.0,79.0,52.0,70.0,83,Right
123,Brentford,Ivan Toney,ST,High/High,78.0,79.0,68.0,74.0,50.0,82.0,81,Right
125,Everton,Dominic Calvert-Lewin,ST,Medium/Medium,81.0,76.0,64.0,74.0,39.0,79.0,81,Right


In [ ]:
candidates.to_csv('D:/Downloads/Fifa Master data/candidates.csv',index=False)
report.to_csv('D:/Downloads/Fifa Master data/report.csv',index=False)